# carlos dataset collection for WCST twin net
 borrows kara's program to make cards

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import random
import itertools
import torch

from numpy.random import default_rng

In [12]:
# Make 1-hot encoded numpy array for each card
#Kara's code generation section
def make_cardList():
    card_list = np.zeros((64,3,4))
    for i, (shape, number, color) in enumerate(itertools.product(range(0,4), range(0,4), range(0,4))):
        card_list[i][0][shape] = 1
        card_list[i][1][number] = 1
        card_list[i][2][color] = 1
        
    return card_list   

def plotCards(card_list):
    f, ax = plt.subplots(nrows=8, ncols=8, figsize=[10,10])
    for i in range(0, card_list.shape[0]):
        ax.flatten()[i].imshow(card_list[i])
    # plt.tight_layout()
    plt.show()     

In [20]:
a = make_cardList()
#plotCards(a)
print(type(a))
a.shape

<class 'numpy.ndarray'>


(64, 3, 4)

# The following are functions I use to make random card lists, which together are used to create custom N-length card dataset with matching indicator  

In [97]:
#make a function that takes N random pairs of cards, regardless if they match or not

def make_randomCardVector(setLength):
    cards = make_cardList()
    set1 = []
    # cards is 64 cards by 3 dimensions by 4 values
    
    #create randomly shuffled index of cardlist
    rng = default_rng() #instance the RNG
    
    indexes = np.arange(cards.shape[0]) #create vector of ALL possible card indexes 
    rng.shuffle(indexes) #shuffles original variable
    
    setIndexes = indexes[0:setLength] #extract first N values of shuffled index array for desired set length
    
    set1 = [cards[i] for i in setIndexes]
    set1 = np.array(set1)
    
    return set1, setIndexes

In [204]:
def make_randomCardPairs(pairSetLength):
# take 2 random cards with replacement 
 #output a stacked cardset: setlength x vector N x 3 x 4
    cardVec1, v1shuffle = make_randomCardVector(pairSetLength)
    cardVec2, v2shuffle = make_randomCardVector(pairSetLength)

    cardDoubleVec = np.zeros((pairSetLength,2,3,4))
    
    cardDoubleVec[:,0,:,:] = cardVec1
    cardDoubleVec[:,1,:,:] = cardVec2
    
    cardDoubleVec = torch.from_numpy(cardDoubleVec)
    return cardDoubleVec    

In [220]:
def eval_cardPair_ruleMatch(cardPair, ruleNum):
#create 3 row vector for each pair detailing if they match on axis 1-3 (represented by 1/0)
    setLength = (cardPair.shape[0]) 
    rulematch = []
    rulematch = [1  if np.array_equal(cardPair[i,0,ruleNum-1,:], cardPair[i,1,ruleNum-1,:]) else 0 for i in np.arange(0,setLength)]
    rulematch = np.array(rulematch)
    
    rulematch = torch.from_numpy(rulematch)
    
    return rulematch #,setLength

In [365]:
test = make_randomCardPairs(10)

print(test.shape)
print(type(test))
testmatch = eval_cardPair_ruleMatch(test,1)
print(type(testmatch), testmatch.shape)
#, setL 
print(testmatch)

torch.Size([10, 2, 3, 4])
<class 'torch.Tensor'>
<class 'torch.Tensor'> torch.Size([10])
tensor([0, 0, 1, 0, 1, 0, 0, 1, 0, 1], dtype=torch.int32)


In [381]:
class randomCardPairSet(torch.utils.data.Dataset):
    def __init__(self, PairSetLength):
        
        remainder = PairSetLength % 64
        fullSetsNeeded = np.floor(PairSetLength/64)
        additionalSets = fullSetsNeeded - 1
        fullSet = 64
        
        self.cardset = torch.empty((1,2,3,4), dtype = torch.double) #need to make this double
        
        if remainder > 0:
            self.cardset = torch.cat((self.cardset, make_randomCardPairs(remainder)))
            
        if fullSetsNeeded > 0:
            self.cardset =  torch.cat((self.cardset, make_randomCardPairs(fullSet)))
        
        if additionalSets > 0:
            for i in np.arange(additionalSets):
                self.cardset = torch.cat((self.cardset, make_randomCardPairs(fullSet)),0)

        self.cardset = self.cardset[1:] #remove placeholder entry
        
        self.match1 = eval_cardPair_ruleMatch(self.cardset, 1)
        self.match2 = eval_cardPair_ruleMatch(self.cardset, 2)
        self.match3 = eval_cardPair_ruleMatch(self.cardset, 3)
    
    #need to remove empty initialization 
    
    def __len__(self):
        return len(self.cardset)
    
    def shape(self):
        return self.cardset.shape
    
    def __getitem__(self,index):
        
        sample = {'cards': self.cardset[index,:,:,:],
                  'match1': self.match1[index],
                  'match2':self.match2[index], 
                  'match3': self.match3[index]}    
        return sample  

### Dataset creation test section

In [376]:

testSet = randomCardPairSet(171)

In [377]:
testSet

In [380]:
print(testSet.shape())

torch.Size([171, 2, 3, 4])


In [345]:
x = 171
print(x, x/64)
remainder = x % 64
print(remainder)
fullSetsNeeded = np.floor(x/64)
print(fullSetsNeeded)
additionalSets = fullSetsNeeded - 1
print(additionalSets)

171 2.671875
43
2.0
1.0


In [386]:
lowTest = randomCardPairSet(24)
lowTest

print(lowTest.shape())

torch.Size([24, 2, 3, 4])


In [383]:
fullTest = randomCardPairSet(64)
print(fullTest.shape())

torch.Size([64, 2, 3, 4])


In [384]:
doubletest = randomCardPairSet(200)
print(doubletest.shape())

torch.Size([200, 2, 3, 4])


# Now I create dataLoader object for training

In [388]:
train_set = randomCardPairSet(10000)
print(train_set.shape())

torch.Size([10000, 2, 3, 4])


In [390]:
batch_size = 128 # you can play around with this
train_loader = torch.utils.data.DataLoader(train_set,
                                           batch_size=batch_size, 
                                           shuffle=True,
                                           num_workers=8)

In [391]:
train_loader

In [392]:
print(train_loader)